# Intro to Recommender Systems Lab

Complete the exercises below to solidify your knowledge and understanding of recommender systems.

For this lab, we are going to be putting together a user similarity based recommender system in a step-by-step fashion. Our data set contains customer grocery purchases, and we will use similar purchase behavior to inform our recommender system. Our recommender system will generate 5 recommendations for each customer based on the purchases they have made.

In [1]:
import pandas as pd
from scipy.spatial.distance import pdist, squareform

In [2]:
data = pd.read_csv('../data/customer_product_sales.csv')

In [3]:
data.head()

,CustomerID,FirstName,LastName,SalesID,ProductID,ProductName,Quantity
0,61288,Rosa,Andersen,134196,229,Bread - Hot Dog Buns,16
1,77352,Myron,Murray,6167892,229,Bread - Hot Dog Buns,20
2,40094,Susan,Stevenson,5970885,229,Bread - Hot Dog Buns,11
3,23548,Tricia,Vincent,6426954,229,Bread - Hot Dog Buns,6
4,78981,Scott,Burch,819094,229,Bread - Hot Dog Buns,20


In [4]:
data.shape

(68584, 7)

In [5]:
#data[(data['CustomerID']==98200) & (data['ProductID']>=392)]

## Step 1: Create a data frame that contains the total quantity of each product purchased by each customer.

You will need to group by CustomerID and ProductName and then sum the Quantity field.

In [6]:
s1 = data[['CustomerID','ProductName','Quantity']].groupby(['CustomerID','ProductName']).sum().reset_index()
s1.head()

,CustomerID,ProductName,Quantity
0,33,Apricots - Dried,1
1,33,Assorted Desserts,1
2,33,Bandage - Flexible Neon,1
3,33,"Bar Mix - Pina Colada, 355 Ml",1
4,33,"Beans - Kidney, Canned",1


## Step 2: Use the `pivot_table` method to create a product by customer matrix.

The rows of the matrix should represent the products, the columns should represent the customers, and the values should be the quantities of each product purchased by each customer. You will also need to replace nulls with zeros, which you can do using the `fillna` method.

In [7]:
s2 = pd.pivot_table(s1, index='ProductName', columns='CustomerID', values='Quantity',aggfunc=sum)
s2 = s2.fillna(0)
#s2.head()
s2.shape


(452, 1000)

## Step 3: Create a customer similarity matrix using `squareform` and `pdist`. For the distance metric, choose "euclidean."

In [8]:
s3_sqr = squareform(pdist(s2.T, metric='euclidean'))
s3_sqr.shape

(1000, 1000)

In [57]:
s3_sqr

array([[  0.        ,  11.91637529,  10.48808848, ..., 228.62851966,
        239.        , 229.77380181],
       [ 11.91637529,   0.        ,  11.74734012, ..., 228.01096465,
        239.03765394, 229.70415756],
       [ 10.48808848,  11.74734012,   0.        , ..., 228.08112592,
        238.26665734, 229.77380181],
       ...,
       [228.62851966, 228.01096465, 228.08112592, ...,   0.        ,
        304.13812651, 305.16389039],
       [239.        , 239.03765394, 238.26665734, ..., 304.13812651,
          0.        , 303.10889132],
       [229.77380181, 229.70415756, 229.77380181, ..., 305.16389039,
        303.10889132,   0.        ]])

## Step 4: Check your results by generating a list of the top 5 most similar customers for a specific CustomerID.

In [9]:
clientes = s2.columns
clientes

Int64Index([   33,   200,   264,   356,   412,   464,   477,   639,   649,
              669,
            ...
            97697, 97753, 97769, 97793, 97900, 97928, 98069, 98159, 98185,
            98200],
           dtype='int64', name='CustomerID', length=1000)

In [61]:
selected = 97769
lista = list(zip(s3_sqr[list(clientes).index(selected)],clientes))
first5customers = sorted(lista, reverse=True, key= lambda x: x[0])[:5]
s4 = [x[1] for x in first5customers]
s4

[96524, 96560, 91777, 92139, 87920]

## Step 5: From the data frame you created in Step 1, select the records for the list of similar CustomerIDs you obtained in Step 4.

In [19]:
s5 = s1[s1.CustomerID.isin(s4)]
s5.head()

,CustomerID,ProductName,Quantity
59268,91443,Appetizer - Sausage Rolls,24
59269,91443,Baking Powder,24
59270,91443,Banana Turning,24
59271,91443,Bananas,24
59272,91443,Bandage - Flexible Neon,24


## Step 6: Aggregate those customer purchase records by ProductName, sum the Quantity field, and then rank them in descending order by quantity.

This will give you the total number of each product purchased by the 5 most similar customers to the customer you selected in order from most purchased to least.

In [30]:
s6 = s5[['ProductName','Quantity']].groupby('ProductName').sum()
s6 = s6.sort_values('Quantity', ascending=False)
s6

,Quantity
ProductName,
"Turnip - White, Organic",122
Pastry - Butterscotch Baked,122
Muffin Chocolate Individual Wrap,98
Clam Nectar,98
Kellogs Special K Cereal,97
...,...
Eggplant - Asian,24
Fond - Neutral,24
Garbag Bags - Black,24


## Step 7: Filter the list for products that the chosen customer has not yet purchased and then recommend the top 5 products with the highest quantities that are left.

- Merge the ranked products data frame with the customer product matrix on the ProductName field.
- Filter for records where the chosen customer has not purchased the product.
- Show the top 5 results.

In [50]:
s7 = s6.merge(s2, how='left', on='ProductName')
s7 = s7[s7[selected]==0].head(5).reset_index()
s7['ProductName']

0             Turnip - White, Organic
1         Pastry - Butterscotch Baked
2    Muffin Chocolate Individual Wrap
3                         Clam Nectar
4            Kellogs Special K Cereal
Name: ProductName, dtype: object

## Step 8: Now that we have generated product recommendations for a single user, put the pieces together and iterate over a list of all CustomerIDs.

- Create an empty dictionary that will hold the recommendations for all customers.
- Create a list of unique CustomerIDs to iterate over.
- Iterate over the customer list performing steps 4 through 7 for each and appending the results of each iteration to the dictionary you created.

In [70]:
recomm = dict()
customers = list(clientes)
for c in customers:
    s8 = list(zip(s3_sqr[customers.index(c)],clientes))
    s8 = sorted(s8, reverse=True, key= lambda x: x[0])[:5]
    s8 = [x[1] for x in s8]
    s8 = s1[s1.CustomerID.isin(s8)]
    s8 = s8[['ProductName','Quantity']].groupby('ProductName').sum()
    s8 = s8.sort_values('Quantity', ascending=False)
    s8 = s8.merge(s2, how='left', on='ProductName')
    s8 = s8[s8[selected]==0].head(5).reset_index()
    recomm[c] = list(s8['ProductName'])

##  Step 9: Store the results in a Pandas data frame. The data frame should a column for Customer ID and then a column for each of the 5 product recommendations for each customer.

In [73]:
s9 = pd.DataFrame.from_dict(recomm, orient='index')
s9.tail(10)

,0,1,2,3,4
97697,Beef - Inside Round,Whmis - Spray Bottle Trigger,Ecolab - Lime - A - Way 4/4 L,Milk - 2%,Spice - Peppercorn Melange
97753,Pastry - Butterscotch Baked,Whmis - Spray Bottle Trigger,"Pasta - Penne, Rigate, Dry",Pastry - Raisin Muffin - Mini,Coffee - Irish Cream
97769,Pork - Hock And Feet Attached,Spice - Peppercorn Melange,Olives - Kalamata,Cup - Translucent 7 Oz Clear,Rum - Mount Gay Eclipes
97793,Whmis - Spray Bottle Trigger,Bagel - Plain,Beef - Inside Round,"Coconut - Shredded, Sweet",Tahini Paste
97900,Cookies - Assorted,Muffin Chocolate Individual Wrap,Pastry - Butterscotch Baked,Squid U5 - Thailand,Table Cloth 81x81 White
97928,Spinach - Baby,Ecolab - Lime - A - Way 4/4 L,"Garlic - Primerba, Paste",Muffin Chocolate Individual Wrap,Pastry - Butterscotch Baked
98069,Tahini Paste,Cheese - Taleggio D.o.p.,"Coconut - Shredded, Sweet",Coffee - Irish Cream,Onions - Vidalia
98159,"Coconut - Shredded, Sweet",Pastry - Butterscotch Baked,Spice - Peppercorn Melange,Soup Campbells - Italian Wedding,Beef - Inside Round
98185,Bread - Rye,Spinach - Baby,Chinese Foods - Chicken,Remy Red,Tray - 16in Rnd Blk
98200,Pastry - Butterscotch Baked,Cheese - Taleggio D.o.p.,"Yogurt - Blueberry, 175 Gr",Soup Campbells - Italian Wedding,Coffee - Irish Cream


## Step 10: Change the distance metric used in Step 3 to something other than euclidean (correlation, cityblock, consine, jaccard, etc.). Regenerate the recommendations for all customers and note the differences.

In [75]:
s3_altern = squareform(pdist(s2.T, metric='correlation'))

In [76]:
recomm10 = dict()
for c in customers:
    s10 = list(zip(s3_altern[customers.index(c)],clientes))
    s10 = sorted(s10, reverse=True, key= lambda x: x[0])[:5]
    s10 = [x[1] for x in s10]
    s10 = s1[s1.CustomerID.isin(s10)]
    s10 = s10[['ProductName','Quantity']].groupby('ProductName').sum()
    s10 = s10.sort_values('Quantity', ascending=False)
    s10 = s10.merge(s2, how='left', on='ProductName')
    s10 = s10[s10[selected]==0].head(5).reset_index()
    recomm10[c] = list(s10['ProductName'])

In [77]:
alternative_df = pd.DataFrame.from_dict(recomm10, orient='index')
alternative_df.tail(10)

,0,1,2,3,4
97697,Tea - Herbal Sweet Dreams,"Crab - Dungeness, Whole",Juice - Apple Cider,Beans - Kidney White,Crush - Cream Soda
97753,"Seedlings - Mix, Organic",Mussels - Cultivated,Sea Bass - Whole,"Mushrooms - Black, Dried",Meldea Green Tea Liquor
97769,Spice - Peppercorn Melange,Lettuce - Treviso,Hersey Shakes,"Pasta - Penne, Rigate, Dry",Dc - Frozen Momji
97793,Flour - Teff,Pomello,Sun - Dried Tomatoes,Chicken - Wieners,Bagel - Plain
97900,Sponge Cake Mix - Chocolate,Ocean Spray - Ruby Red,Berry Brulee,Muffin - Zero Transfat,Veal - Eye Of Round
97928,Wine - Crozes Hermitage E.,Pork - Belly Fresh,Smoked Paprika,"Oranges - Navel, 72",Wiberg Super Cure
98069,"Oil - Shortening,liqud, Fry",Mustard - Seed,Wine - Valpolicella Masi,Sardines,Soupfoamcont12oz 112con
98159,Sponge Cake Mix - Chocolate,Zucchini - Yellow,Pastry - Butterscotch Baked,Langers - Ruby Red Grapfruit,Squid U5 - Thailand
98185,Cocoa Butter,Anchovy Paste - 56 G Tube,"Lamb - Whole, Fresh",Bread - Rye,Tray - 16in Rnd Blk
98200,Scallops 60/80 Iqf,"Beef - Tenderlion, Center Cut",Campari,"Fish - Scallops, Cold Smoked",Corn Meal
